# 1. Tiedonkeruu ja esikäsittely

## 1.1 Rajapinnat

VOOKA-projektin ETL-työkalussa tietojen kerääminen on mahdollista teknisesti kolmesta eri lähteestä:

1. OGC:n WFS-standardin mukaisista rajapinnoista,
2. Esrin ArcGIS Feature Layereista sekä
3. OGC API Features -rajapinnoista

Kaksi ensimmäiseksi mainittua palauttavat syötetyn URL:n mukaisen tiedon kokonaisuudessaan GeoPandas GeoDataframena.

### 1.1.1 WFS ja ArcGIS Feature Layer

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
from lib.get_feature_data import getWFSlayers, getWFSdata, getArcgisFeatureLayer

In [ ]:
# Hae tasot ja data
print(getWFSlayers(url="<insert layers-url here>"))
yk_data = getWFSdata(url="insert wfs-data-url here", layer="<insert layer name here>")
print(type(yk_data))

# Tallennus
yk_data.to_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

In [ ]:
# Esimerkki 2.
rak_data = getArcgisFeatureLayer(layer_url="<insert url here>")
print(type(rak_data))

# Tallennus
rak_data.to_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

### 1.1.2 OGC API Features

OGC API Features -skripti palauttaa API:n sisältöä GeoJSON-formaatissa Etelä-Savon maankunnan rajauksella (bounding box), joka on kovakoodattu toteutukseen. **Jatkokehityksessä kovakoodattu maantieteellinen rajaus tulee korvata halutun alueen koordinaateilla!**

Skiptiin on myös lisätty apufunktioita, joiden avulla MML:n kiinteistötietorajapinnasta haettu GeoJSON-muotoinen dataskeema voidaan normalisoida tabulaariseksi tiedoksi GeoPandas GeoDataframeen ja tallentaa pickle-kirjaston pkl-tiedostona.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.OGC_API_Features_handler import getDataFromAPI, normalizeKiinteistorajaSchema, normalizeKiinteistotunnusSchema, normalizeRajamerkkiSchema

In [ ]:
# Esimerkki 3.
# Kirjoita parametreihin halutut tiedot (lisätietoja OGC_API_Features_handler.py funktioiden docstringeista)
# Tarvitset kiinteistötunnustietoja (piste) sekä kiinteistöpalstatietoja (polygon) myöhemmin!
api_data = getDataFromAPI(url="<insert url here>",
                          username="<insert username here>",
                          password="<insert password here>",
                          outfp=r"<insert output filepath here>.pkl")

# MML:n kiinteistötietorajapinnan tietojen normalisointi ja pkl-tallentaminen
norm_data_1 = normalizeKiinteistotunnusSchema(infp=r"<insert input filepath here>.pkl", outfp=r"<insert output filepath here>.pkl")
#norm_data_2 = normalizeKiinteistorajaSchema(infp="<insert input filepath here>", outfp="<insert output filepath here>")
#norm_data_3 = normalizeRajamerkkiSchema(infp="<insert input filepath here>", outfp="<insert output filepath here>")

## 1.2 Tiedonkeruu

Tällä koodilla voidaan ladata suoraan CSV-taulukkoon koostetusta url-osoitetaulukosta tiedostoja. Tätä varten tarvitaan vain CSV-taulukko, joka sisältää listan niistä tiedostoista, jotka halutaan ladata.

Koodi ilmoittaa mikäli tiedoston lataaminen epäonnistui.

Pohjois-Savon VOOKA-pilotissa tällä koodilla kerättiin kaava-asiakirjoja.

**HUOM! Lataa tiedostoja vain sellaisista osoitteista, jotka ovat luotettavia.**

In [ ]:
#CSV-tiedosto, jossa url-lista ladattavista dokumenteista. Sarakkeen nimeksi laitetaan "osoite".
csv_file = r'<insert filepath here>'

#Sarakkeen nimi, jossa url-lista sijaitsee
column_name = '<insert column name>'

#Valitse kansio, jonne haluat dokumenttien latautuvan.
download_dir = r'<insert filepath here>'

%run -i lib/download_files_from_url_main.py

## 1.3 Esikäsittely

### 1.3.1 KTJ-aineisto

MML:n KTJ-aineistoissa kuntaliitosalueiden kaavoille on ilmoitettu vanha kuntakoodi. Nämä tulee päivittää vastaamaan voimassa olevaa kuntakoodia kattavan vertailun mahdollistamiseksi kunta-aineistojen kanssa. **Toteutus on koodattu vastaamaan ainoastaan Etelä-Savon maakuntien kuntakoodeja!** Input parametrina käytetään GeoPandas GeoDataframe muodossa olevaa KTJ-aineistoa.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.ktj_update_kuntakoodit import updateOldKuntakoodi

In [ ]:
# Lisää tiedostopolut KTJ-aineiston input ja output geopackageen sekä karttataso
ktj_data = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")
ktj_update = updateOldKuntakoodi(ktj_data)

# Tallennus
ktj_update.to_file(r"<insert filepath here>.gpkg", layer="<insert layer name here>")

### 1.3.2 Kunta-aineistot

Kunta-aineistojen esikäsittelytarpeet voivat olla hyvin kuntaspesifejä. VOOKA-projektin tapauksessa erään kunnan WFS-rajapinta oli toteutettu KuntaGML-formaatissa, jonka sisäänluku ei onnistunut perinteisin menetelmin. Ongelma ratkaistiin erillisellä XML-parser-skriptillä, jossa kaavatiedot irrotettiin suoraan merkintäkielen rakenteesta.

Lisäksi osa kunta-aineistoista voi olla saatavina vain erillistoimitettuina CAD-piirroksina. VOOKA-projetissa erään kunnan CAD-aineistossa ominaisuustiedot oli sidottu pistegeometrioihin varsinaisten kaavarajojen sijasta. Ominaisuustietojen yhdistäminen kaavarajoihin toteutettiin hankkeessa niin ikään erillisellä skriptillä, jota edelsi tiedon muunnos paikkatietoformaattiin FME-ohjelmistolla.

Näihin tapauksiin voi käydä tutustumassa erikseen VOOKA-hankkeen GitHub-sivujen kautta. Erillisskriptit löytyvät [muut-kansiossa](https://github.com/ubigu/vooka/tree/master/muut).

## 1.4 PDF-linkityskonversio

### 1.4.1 Linkitystaulu

PDF-kaava-asiakirjojen linkittäminen yhtenäisessä muodossa kaavojen paikkatietoihin on mahdollista, jos tiedetään, mihin kaavaindeksitunnukseen kukin liite kytkeytyy. Tässä ETL-työkalussa esitetyt automatisoinnit vaativat validointia, eli linkittäjän tuottama aineisto tulee tarkistaa osaltaan manuaalisesti. Lisäksi automatisaation avulla ei voida poistaa tarvetta manuaaliselle linkittämiselle.

Työssä PDF-linkitystä varten koostettiin liitostaulu, jossa jokaisella rivillä on tieto kaavan indeksitunnuksesta (KTJ, kunnan aineisto tai molemmat) sekä kaavan dokumenttityypistä (esim. osallistamis- ja arviointisuunnitelma). Kaiken kaikkiaan liitostaulun skeema oli seuraavanlainen:

| Ominaisuustieto | Selite |
| :--- | :--- |
| Kuntanumero | Kunnan virallinen kuntakoodi |
| Kunnan indeksitunnus | Kunnan paikkatietomuotoisen kaava-aineiston kaavaindeksitunnus |
| KTJ-indeksitunnus | KTJ-aineiston kaavaindeksitunnus ("kaavatunnus_1") |
| Original filename | Kunnalta saadun kaavaliitteen alkuperäinen tiedostonimi |
| New filename | Sarake uudelle tiedostonimelle|
| Kaavalaji | Kaavalajikoodisto ylätasolla |
| Manuaalisesti tarkistettu | Onko aineisto tarkistettu manuaalisesti (boolean) |
| Dokumentin tyyppi | Dokumenttityyppikoodisto, numeerinen |
| Match equivalency % | Automaattisen linkityksen vastaavuusprosentti |
| Huomioita | Kenttä havaintojen kirjaamista varten |
| Multipage | Onko liitteessä useampi kuin yksi sivu (boolean) |
| Tila | Koodisto, joka kertoo, onko kaavaliite validi |
| Voimassa oleva | Onko liite voimassa vai ei (boolean) |
| Geometry origin | Geometrian lähdeaineisto |
| Virhetyyppi | Kuvaus mahdollisesta havaitusta virheestä, numeerinen |
| Kuvaus | Kaavatietomallia varten geometry origin ja virhetyyppi -kentistä generoitu kuvaava teksti |

Kaavalaji-koodisto:
- ak
- rak
- yk

Dokumentin tyyppi -koodisto (sanallinen ja numeerinen):
- 1 = kaavakartta (sis. merkinnät ja määräykset)
- 2 = kaavakartta (ei sis. merkinnät ja määräykset)
- 3 = merkinnät ja määräykset (erillisenä)
- 6 = muu

Tila-koodisto:
- ok
- ei ok

Geometry origin-koodisto:
- kunta
- KTJ
- digitoitu VOOKA:ssa
- ei geometriaa

Virhetyyppi-koodisto:
- 0 = ei virhettä
- 1 = Virhe/puute asiakirjassa
- 2 = Virhe/puute rajauksessa
- 3 = Virhe/puute asiakirjassa sekä rajauksessa
- 4 = Muu virhe/puute
- 5 = Ei indeksitunnusta


### 1.3.2 Kaava-asiakirjojen hakemistorakenne

Kunnilta saadut kaavaliitteet tallennettiin vakioituun hakemistorakenteeseen resurssienhallintaan. Hakemistorakenteen runko oli muotoa:

```bash
documents   
│
└───kuntakoodi
│   │   
│   │
│   └───ak
│   │   asiakirja.pdf
│   │   asiakirja2.pdf
│   │   ...
│   └───rak
│   │   asiakirja.pdf
│   │   asiakirja2.pdf
│   │   ...
│   └───yk
│       asiakirja.pdf
│       asiakirja2.pdf
│       ...
...
```

**PDF-linkityskonversion ETL-funktiot on rakennettu kyseisen hakemistorakenteen varaan!**

### 1.4.4 Kaava-asiakirjojen lajittelu hakemistorakenteen mukaan kansioihin

Tämä koodi lajittelee PDF-asiakirjat hakemistorakenteen mukaisesti. Koodi olettaa, että asiakirjat on lajiteltu kunnan nimellä olevaan kansioon kaavalajeittain. Lähtötilanteessa aineiston tulisi olla jaettuna asemakaavoihin, ranta-asemakaavoihin sekä yleiskaavoihin. Tämän enempää lajittelua ei ole tarpeen tehdä. Mikäli tätä lajittelua ei ole tehty, koodi antaa virheen.

Tarkoituksena on välttää kansioiden alikansioiden avaamisesta syntyvää lisätyötä sekä automaattisesti poimia vain PDF-muotoiset asiakirjat.

In [ ]:
valittu_kansio = r'<insert filepath here>'


%run -i lib/sort_files_by_kaavalaji_main.py

### 1.4.5 Kaavaindeksien kansiolajittelu

Tämä koodi lajittelee kaavaindeksiaineistot (.gpkg, .shp, .dwg, .dxf) hakemistorakenteen mukaisesti. Koodi olettaa, että asiakirjat on lajiteltu kunnan nimellä olevaan kansioon kaavalajeittain. Lähtötilanteessa aineiston tulisi olla jaettuna asemakaavoihin, ranta-asemakaavoihin sekä yleiskaavoihin. Tämän enempää lajittelua ei ole tarpeen tehdä. Mikäli tätä lajittelua ei ole tehty, koodi antaa virheen.

Tarkoituksena on välttää kansioiden alikansioiden avaamisesta syntyvää lisätyötä sekä automaattisesti poimia vain indeksiaineistot.

In [ ]:
valittu_kansio = r'<insert filepath here>'


%run -i lib/sort_index_files_by_kaavalaji_main.py

### 1.4.6 Kaavatunnuksella nimettyjen asiakirjojen linkitys kaavatunnukselliseen kunnan indeksiaineistoon

<span style="color:red">**Tämä vaihe ei ole pakollinen! Vaatii kelvollisen paikkatietoaineiston, joka sisältää kunnan kaavatunnuksen. Lisäksi kunnan toimittamien asiakirjojen tiedostonimien tulee sisältää kaavatunnus.**</span>

Mikäli kaava-asiakirjojen nimet sisältävät kunnan kaavatunnuksen ja kunnasta on toimitettu paikkatietoaineisto, joka sisältää kaavatunnuksen, on mahdollista linkittää kaava-asiakirjat indeksi aineistoon automaattisesti.

HUOM!
Automatisaatio perustuu sumeaan logiikkaan tarkoittaen sitä, että automatisaation linkittämät indeksit eivät välttämättä vastaa toisiaan täydellisesti.

Automatisaatio tuottaa CSV-taulukkoon arvon, joka ilmoittaa kuinka hyvin tiedostonimi vastasi indeksiä. Jos arvo on alle 80, niin tilanne tulee arvioida tilannekohtaisesti. Mitä matalampi arvo on, sitä todennäköisemmin automatisaatio on antanut väärän tai valheellisen linkitystuloksen.

HUOM!

Koodi tulkitsee dokumentin tyypin tiedostonimestä. Koodi antaa oletuksena dokumentin tyypiksi kaavakartan määräyksineen (arvo 1). Mikäli nimessä mainitaan merkintä tai määräys koodi antaa dokumentin tyypiksi kaavamääräyksen (3).
NÄMÄ TULEE AINA TARKISTAA MANUAALISESTI!

(Usein kaava-asiakirjojen tiedostonimessä voi esiintyä määräykseen viittaava sana, mutta todellisuudessa asiakirja sisältää sekä kartan että määräykset: esim. Kaava_1_kartta_merkinnat.pdf)

Koodin suorittamiseksi tarvitset kansion, jossa asiakirja-aineisto sijaitsee. Lisäksi tarvitset paikkatietomuotoisen indeksiaineiston.

Koodin suorittamiseksi anna:

-sen kunnan kuntanumero, jota haluat käsitellä.
-mitä kaavalajia olet käsittelemässä.
-missä sarakkeessa indeksitunnus sijaitsee paikkatiedossa.

Lisäksi anna koodiin se kansio, johon haluat valmiin CSV-tiedoston.

In [ ]:
#Anna tiedostopolku kansioon missä kaava-asiakirjat sijaitsevat

kaava_asiakirjat = r'<insert filepath here>'


#Anna tiedostopolku tiedostonimineen kunnan indeksiaineistoon

#Indeksiaineiston tulee olla .gpkg tai .shp

kunta_kaavaindeksi = r'<insert filepath here>'

#Anna kuntanumero

kunta_numero = '<insert kuntanumero here>'

#Anna kaavalajin nimi

kaavalaji = '<insert kaavalaji here>'

#Anna sarakkeen nimi, jossa indeksitieto sijaitsee
indeksi_sarake = '<insert column name here>'

#Laita tähän sen kansion polku, johon haluat saada kaavatiedot.
directory = r'<insert kaavalaji here>'

#Suorita linkitysfunktio

%run -i lib/matching_filename_to_kaavaindex_main.py

Tuotoksena syntyy CSV-taulukko, joka vastaa PDF-linkityskonversiotaulukkoa.

| Kuntanumero | Kunnan indeksitunnus | KTJ-indeksitunnus | Original filename | New filename | Match equivalency %  | Kaavalaji | Manuaalisesti tarkistettu | Dokumentin tyyppi |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| "Kuntanumero" | "Kunnan kaavaindeksi" | "Tyhjä" | "Tiedoston nimi" | "Tyhjä" | "Vertailuarvo" | "Kaavalaji" | "FALSE" | "Dokumentin tyyppi" |
| 140* | 1* | - | Kaava_1.pdf | - | 90 | ak* | FALSE | 1* |

*esimerkki

### 1.4.7 Kaava-asiakirjataulun luominen PDF-linkityskonversiotauluun

<span style="color:red">**Mikäli vaihetta 1.4.6 ei voitu suorittaa aineistolle, voidaan suorittaa asiakirjataulun luonti tauluun ilman tiedostonimien linkitystä paikkatietoaineistoon.**</span>

Koodilla voidaan tuottaa useamman kunnan kaava-asiakirjat lajiteltuna taulukkoon.

HUOM! Tämä vaatii sen, että kohdan 1.3.2 ohjetta hakemistorakenteeseen on noudatettu.

In [ ]:
#Anna tiedostopolku kansioon missä kaava-asiakirjat sijaitsevat
path_to_folder = r'<insert filepath here>'


#Laita tähän sen kansion polku, johon haluat saada kaavatiedot.
file_path = r'<insert filepath here>/'

%run -i lib/write_csv_main.py